In [1]:
from srai.regionalizers import geocode_to_region_gdf
from srai.regionalizers import AdministrativeBoundaryRegionalizer
from esda.moran import Moran
from libpysal.weights import Queen, KNN
import seaborn
import pandas
import geopandas
import numpy
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
CITY = "Wrocław" 
COUNTRY = "Poland"

area_name = f"{CITY}, {COUNTRY}"
area = None

area = geocode_to_region_gdf(area_name)

regions = AdministrativeBoundaryRegionalizer(9).transform(area)

Loading boundaries: 48: 100%|██████████| 1/1 [00:00<00:00,  2.29it/s]
c:\Users\Todd Howard\Desktop\Semester 2\Dane przestrzenne\spatial-project\.venv\lib\site-packages\srai\regionalizers\administrative_boundary_regionalizer.py:165: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  points_collection: Optional[BaseGeometry] = gdf_wgs84[


In [3]:
regions.explore()

In [4]:
import os
import sys
import os
if os.path.basename(os.getcwd()) != 'spatial-project':
    os.chdir(os.path.dirname(os.getcwd()))

from src.data_loading import load_places_properties, load_places

geo_properties = load_places_properties().drop(index=0)

geoplaces_raw = load_places(as_geo=True)
variables = ['name', 'place_id', 'price_level', 'rating', 'user_ratings_total', 'geometry'
            #  , 'latitude', 'longitude'
             ]
geoplaces = geoplaces_raw[variables].fillna(-1)

In [5]:
geo_properties['place_id'].value_counts().head(3)

place_id
ChIJeccWamjBD0cRr8l49FmA3sI    2
ChIJo1BK_-bDD0cRf5vk8Qaul2o    1
ChIJswhtXZLnD0cRNRlkm7hJBg4    1
Name: count, dtype: int64

In [6]:
geo_properties = geo_properties.set_index('place_id').drop(index='ChIJeccWamjBD0cRr8l49FmA3sI').reset_index()

In [7]:
geo_united = geoplaces.set_index('place_id').join(geo_properties.set_index('place_id'), how='inner').reset_index()

In [8]:
len(geo_united)

649

In [9]:
df_list = []

for root, folders, files in os.walk(os.path.join('data', 'Dane sentyment')):
    for f in files:
        full_f = os.path.join(root, f)
        df_list.append(pd.read_csv(full_f))
total_df = pd.concat(df_list)

places_embed = total_df[['place_id', 'sentiment_score', 'n_stars']].groupby(by='place_id').mean()

geo_united = geo_united.join(places_embed, on='place_id')

In [11]:
# geo_united.columns

In [21]:
anal_vars = ['price_level', 
             'rating', 
             'user_ratings_total',
            #  'Dostawa', 
            #  'Dostawa bez kontaktu', 
            #  'Na miejscu', 
            #  'Na wynos',
            #  'Odbiór na zewnątrz', 
            #  'Ogródek',
            #  'Dobre miejsce na przyjęcia urodzinowe dla dzieci',
            #  'Krzesełka do karmienia', 
            #  'Menu dla dzieci', 
            #  'Odpowiednie dla dzieci',
            #  'Dla całej rodziny', 
            #  'Miejsce przyjazne dla osób transpłciowych',
            #  'Przyjazne dla osób LGBTQ+', 
            #  'Przytulne', 
            #  'Swobodna atmosfera',
            #  'Dobre drinki', 
            #  'Szybka obsługa', 
            #  'Desery', 
            #  'Kolacja', 
            #  'Obiad',
            #  'Późne śniadanie', 
            #  'Śniadanie', 
            #  'Alkohol', 
            #  'Kawa', 
            #  'dania wegańskie',
            #  'dania wegetariańskie', 
            #  'Piwo', 
            #  'Wino', 
            #  'Koktajle', 
            #  'Bezpłatny parking',
            #  'Możliwa rezerwacja', 
            #  'Bar na miejscu', 
            #  'Bezpłatne Wi-Fi',
            #  'Można przyjść z psem', 
            #  'Toaleta', 
            #  'Miejsca dla osób na wózkach',
            # 'sentiment_score', 
            # 'n_stars',
             ]

In [36]:

joint_hex = None
### BEGIN SOLUTION
from srai.joiners import IntersectionJoiner
joint = IntersectionJoiner().transform(regions, geo_united.set_index('place_id'))

reg_with_stuff = joint.reset_index()['region_id'].unique()
regions_filtered = regions[regions.index.isin(reg_with_stuff)]

from srai.embedders import GTFS2VecEmbedder as embedderClass

features = geo_united[['place_id'] + anal_vars].set_index('place_id')
features.index.names = ['feature_id']


embeddings = joint.join(features, on='feature_id').reset_index('feature_id').drop(columns=['feature_id']).groupby(by='region_id').mean()

from sklearn.preprocessing import robust_scale

geo_scale = robust_scale(embeddings)

regions_filtered = regions_filtered.loc[embeddings.index] # sort regions how it is in embeddings df

w = Queen.from_dataframe(regions_filtered)

from sklearn.cluster import AgglomerativeClustering

numpy.random.seed(0)
# Specify cluster model with spatial constraint
model = AgglomerativeClustering(
linkage="ward", connectivity=w.sparse, n_clusters=5
)
# Fit algorithm to the data
model.fit(geo_scale)

result_gdf = embeddings.copy()
result_gdf['labels'] = model.labels_


from srai.plotting import plot_numeric_data
data_column = "labels"
plot_numeric_data(regions, data_column, result_gdf)


C:\Users\Todd Howard\AppData\Local\Temp\ipykernel_15464\3200017130.py:23: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(regions_filtered)


In [37]:
result_gdf.reset_index(drop=True).groupby(by='labels').mean()

,price_level,rating,user_ratings_total
labels,,,
0,0.506711,4.446977,380.141175
1,0.486593,4.114964,475.788603
2,-0.166667,3.283333,1285.500000
3,1.209302,4.139535,2261.186047
4,0.966877,4.281292,1166.324527


# Sentiment

In [39]:
anal_vars = [
            #  'price_level', 
            #  'rating', 
             'user_ratings_total',
            #  'Dostawa', 
            #  'Dostawa bez kontaktu', 
            #  'Na miejscu', 
            #  'Na wynos',
            #  'Odbiór na zewnątrz', 
            #  'Ogródek',
            #  'Dobre miejsce na przyjęcia urodzinowe dla dzieci',
            #  'Krzesełka do karmienia', 
            #  'Menu dla dzieci', 
            #  'Odpowiednie dla dzieci',
            #  'Dla całej rodziny', 
            #  'Miejsce przyjazne dla osób transpłciowych',
            #  'Przyjazne dla osób LGBTQ+', 
            #  'Przytulne', 
            #  'Swobodna atmosfera',
            #  'Dobre drinki', 
            #  'Szybka obsługa', 
            #  'Desery', 
            #  'Kolacja', 
            #  'Obiad',
            #  'Późne śniadanie', 
            #  'Śniadanie', 
            #  'Alkohol', 
            #  'Kawa', 
            #  'dania wegańskie',
            #  'dania wegetariańskie', 
            #  'Piwo', 
            #  'Wino', 
            #  'Koktajle', 
            #  'Bezpłatny parking',
            #  'Możliwa rezerwacja', 
            #  'Bar na miejscu', 
            #  'Bezpłatne Wi-Fi',
            #  'Można przyjść z psem', 
            #  'Toaleta', 
            #  'Miejsca dla osób na wózkach',
            'sentiment_score', 
            'n_stars',
             ]

In [40]:

joint_hex = None
### BEGIN SOLUTION
from srai.joiners import IntersectionJoiner
joint = IntersectionJoiner().transform(regions, geo_united.set_index('place_id'))

reg_with_stuff = joint.reset_index()['region_id'].unique()
regions_filtered = regions[regions.index.isin(reg_with_stuff)]

from srai.embedders import GTFS2VecEmbedder as embedderClass

features = geo_united[['place_id'] + anal_vars].set_index('place_id')
features.index.names = ['feature_id']


embeddings = joint.join(features, on='feature_id').reset_index('feature_id').drop(columns=['feature_id']).groupby(by='region_id').mean()

from sklearn.preprocessing import robust_scale

geo_scale = robust_scale(embeddings)

regions_filtered = regions_filtered.loc[embeddings.index] # sort regions how it is in embeddings df

w = Queen.from_dataframe(regions_filtered)

from sklearn.cluster import AgglomerativeClustering

numpy.random.seed(0)
# Specify cluster model with spatial constraint
model = AgglomerativeClustering(
linkage="ward", connectivity=w.sparse, n_clusters=5
)
# Fit algorithm to the data
model.fit(geo_scale)

result_gdf = embeddings.copy()
result_gdf['labels'] = model.labels_


from srai.plotting import plot_numeric_data
data_column = "labels"
plot_numeric_data(regions, data_column, result_gdf)


C:\Users\Todd Howard\AppData\Local\Temp\ipykernel_15464\3200017130.py:23: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(regions_filtered)


In [41]:
result_gdf.reset_index(drop=True).groupby(by='labels').mean()

,user_ratings_total,sentiment_score,n_stars
labels,,,
0,410.600185,0.283585,4.324076
1,1006.067236,0.188728,4.118334
2,1285.500000,0.024578,3.610629
3,2261.186047,0.320845,4.189973
4,411.785714,0.491280,4.615449
